# Project - topic modelling + polarity score based on words


Amazon cell phones evaluation platform

I did it for each brand, not for each products, due to the small number of reviews...


In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
import gzip
import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('sentiwordnet')
import string
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rassinoux\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rassinoux\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rassinoux\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\rassinoux\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [2]:
# read the data from previosly created csv files
reviews_df = pd.read_csv('./data/reviews_cellphones.csv')
metadata_df = pd.read_csv('./data/metadata_cellphones.csv')
ratings_cols = ['reviewerID', 'asin', 'overall', 'unixReviewTime']
ratings_df = reviews_df[ratings_cols]

reviews_df.drop('Unnamed: 0', axis=1, inplace=True)
metadata_df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
phones_df = metadata_df[metadata_df.categories.str.
            startswith("[['Cell Phones & Accessories', 'Cell Phones', 'Unlocked ")]
phones_df = phones_df[(pd.notnull(phones_df['brand'])) & (phones_df['brand'] != '')]

gsmarena_df = pd.read_csv('./data/phone_dataset.csv', error_bad_lines=False)

b'Skipping line 821: expected 40 fields, saw 41\nSkipping line 6060: expected 40 fields, saw 41\nSkipping line 6663: expected 40 fields, saw 41\n'


In [4]:
list_brands_accepted = gsmarena_df.brand.str.lower().unique()
phones_df_filtered = phones_df[phones_df['brand'].str.lower().isin(list_brands_accepted)]

In [118]:
final_df = pd.merge(phones_df_filtered, reviews_df, on='asin', how='inner')
print('Number of reviews in total:', len(final_df))
final_df

Number of reviews in total: 56222


,asin,related,title,price,salesRank,imUrl,brand,categories,description,reviewerID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,B0002Z8CSA,"{'also_viewed': ['B007N0RAXK', 'B00BNFLGM2', '...",Motorola V400 Unlocked Cell Phone--U.S. Versio...,81.95,{'Cell Phones & Accessories': 602817},http://ecx.images-amazon.com/images/I/4142320E...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",The quad-band Motorola V400 cell phone feature...,A1LZEEL11U755M,A. Le,"[2, 2]","I didn't buy from this seller, but I owned thi...",5.0,sturdiest phone ever!,1240012800,"04 18, 2009"
1,B0002Z8CSA,"{'also_viewed': ['B007N0RAXK', 'B00BNFLGM2', '...",Motorola V400 Unlocked Cell Phone--U.S. Versio...,81.95,{'Cell Phones & Accessories': 602817},http://ecx.images-amazon.com/images/I/4142320E...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",The quad-band Motorola V400 cell phone feature...,A21BFCRY0I0RD7,"Jose Carlos P. Reyes ""otomed""","[4, 4]",I've been using a Motorola V190 for well over ...,5.0,Good phone for day to day use,1261958400,"12 28, 2009"
2,B0002Z8CSA,"{'also_viewed': ['B007N0RAXK', 'B00BNFLGM2', '...",Motorola V400 Unlocked Cell Phone--U.S. Versio...,81.95,{'Cell Phones & Accessories': 602817},http://ecx.images-amazon.com/images/I/4142320E...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",The quad-band Motorola V400 cell phone feature...,A2ND5GHEMRZV0D,Leonila Romualdo,"[1, 5]",order took 2 weeks because the package came ba...,2.0,motorola from freehasslecell.com,1235433600,"02 24, 2009"
3,B0009FCAJA,"{'also_bought': ['B000IN5ZHM', 'B0016JDE34', '...","Motorola RAZR V3 Unlocked Phone with Camera, a...",69.99,{'Cell Phones & Accessories': 10173},http://ecx.images-amazon.com/images/I/41294T68...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",When the Motorola RAZR V3 was launched last ye...,A36EJJUDZ37UHS,NaN,"[4, 4]",I've ordered 3 of these phones from 2 differen...,1.0,phone is nothing but problems - not authentic ...,1200096000,"01 12, 2008"
4,B0009FCAJA,"{'also_bought': ['B000IN5ZHM', 'B0016JDE34', '...","Motorola RAZR V3 Unlocked Phone with Camera, a...",69.99,{'Cell Phones & Accessories': 10173},http://ecx.images-amazon.com/images/I/41294T68...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",When the Motorola RAZR V3 was launched last ye...,A2T8YGL2PJJ93A,4004CII,"[8, 9]","I read the other reviews, heard about the drea...",2.0,What a piece of junk!,1158710400,"09 20, 2006"
5,B0009FCAJA,"{'also_bought': ['B000IN5ZHM', 'B0016JDE34', '...","Motorola RAZR V3 Unlocked Phone with Camera, a...",69.99,{'Cell Phones & Accessories': 10173},http://ecx.images-amazon.com/images/I/41294T68...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",When the Motorola RAZR V3 was launched last ye...,A5KB2AIDW621E,Aaron,"[0, 0]",Bought this for my mother-in-law. After chargi...,1.0,Phone doesn't hold charge,1333843200,"04 8, 2012"
6,B0009FCAJA,"{'also_bought': ['B000IN5ZHM', 'B0016JDE34', '...","Motorola RAZR V3 Unlocked Phone with Camera, a...",69.99,{'Cell Phones & Accessories': 10173},http://ecx.images-amazon.com/images/I/41294T68...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",When the Motorola RAZR V3 was launched last ye...,A2RJGRJUPYMEJ2,"Adrian Thompson ""Adrian""","[0, 1]",This phone is really nice. I've owned it for a...,4.0,Good for a simple life,1180915200,"06 4, 2007"
7,B0009FCAJA,"{'also_bought': ['B000IN5ZHM', 'B0016JDE34', '...","Motorola RAZR V3 Unlocked Phone with Camera, a...",69.99,{'Cell Phones & Accessories': 10173},http://ecx.images-amazon.com/images/I/41294T68...,Motorola,"[['Cell Phones & Accessories', 'Cell Phones', ...",When the Motorola RAZR V3 was launched last ye...,A1KVW6P17VM6VB,a frequent visitor,"[1, 2]","The phone is great, but when we tried to buy r...",4.0,"phone good, but no web access with Cingular",1186617600,"08 9, 2007"
8,B0009FCAJA,"{'also_bought': ['B000IN5ZHM', 'B0016JDE34', '...","Motorola RAZ

In [34]:
final_df.to_csv('./data/final_dataset.csv')

### SENTIMENT ANALYSIS BASED ON FEATURE WORD FOR A BRAND

In [223]:
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

ratings = np.arange(1, 6, 1)
stop_words = text.ENGLISH_STOP_WORDS.union(['phone', 'phones', 'iphone', 'apple', 'samsung', 'amazon']) # Common words

NUMBER_TOPICS = 10
MAX_ITER = 100
MAX_FEATURES = 250
TOP_WORDS = 10

brands = ['Apple', 'Samsung', 'LG']

df = pd.DataFrame()

for brand in brands:
    for rate in ratings:
        reviews = final_df.query('brand==@brand & overall==@rate')['reviewText'].dropna()
        tf_vectorizer = CountVectorizer(max_features=MAX_FEATURES, stop_words=stop_words)
        tf = tf_vectorizer.fit_transform(reviews)
        tf_feature_names = tf_vectorizer.get_feature_names()

        lda = LatentDirichletAllocation(n_components=NUMBER_TOPICS, max_iter=1, random_state=0, learning_method='batch').fit(tf)

        #Normalize
        lda.components_ /= lda.components_.sum(axis=1)[:, np.newaxis]

        for topic_idx, topic in enumerate(lda.components_):
            temp_df = pd.DataFrame()
            for i in topic.argsort()[:-TOP_WORDS - 1:-1]:
                df = df.append({'brand': brand, 'overall':rate, 'topic_id':topic_idx + 1, 'score': topic[i], 'word':tf_feature_names[i]}, ignore_index=True)


In [224]:
# init sentiment 'analysis'
df["pos_score"] = 0.0
df["neg_score"] = 0.0
df["neu_score"] = 0.0
df["comp_score"] = 0.0

In [225]:
df

,brand,overall,score,topic_id,word,pos_score,neg_score,neu_score,comp_score
0,Apple,1.0,0.037741,1.0,new,0.0,0.0,0.0,0.0
1,Apple,1.0,0.020063,1.0,received,0.0,0.0,0.0,0.0
2,Apple,1.0,0.018439,1.0,stolen,0.0,0.0,0.0,0.0
3,Apple,1.0,0.018343,1.0,item,0.0,0.0,0.0,0.0
4,Apple,1.0,0.016087,1.0,work,0.0,0.0,0.0,0.0
5,Apple,1.0,0.015329,1.0,days,0.0,0.0,0.0,0.0
6,Apple,1.0,0.015156,1.0,return,0.0,0.0,0.0,0.0
7,Apple,1.0,0.014359,1.0,didn,0.0,0.0,0.0,0.0
8,Apple,1.0,0.013649,1.0,seller,0.0,0.0,0.0,0.0
9,Apple,1.0,0.011793,1.0,lost,0.0,0.0,0.0,0.0


Problem: we have not balanced data

In [226]:
feature_words = ["apps", "camera", "price", "battery", "charge", "sound", "memory", "screen", "speaker"]

In [227]:
from nltk import sentiment

scorer = SentimentIntensityAnalyzer()
score = dict()
for row_num in range(0, len(df), 1):
    word = df['word'][row_num]
    
    # Give score only to non feature words
    if not word in feature_words:
        score = scorer.polarity_scores(word)
        pos_score = score['pos'] * df['score'][row_num] * 100
        neg_score = score['neg'] * df['score'][row_num] * 100
        neu_score = score['neu'] * df['score'][row_num] * 100
        comp_score = score['compound'] * df['score'][row_num] * 100

        df.set_value(row_num, 'pos_score', pos_score)
        df.set_value(row_num, 'neg_score', neg_score)
        df.set_value(row_num, 'neu_score', neu_score)
        df.set_value(row_num, 'comp_score', comp_score)

In [228]:
df

,brand,overall,score,topic_id,word,pos_score,neg_score,neu_score,comp_score
0,Apple,1.0,0.037741,1.0,new,0.000000,0.000000,3.774060,0.000000
1,Apple,1.0,0.020063,1.0,received,0.000000,0.000000,2.006291,0.000000
2,Apple,1.0,0.018439,1.0,stolen,0.000000,1.843872,0.000000,-0.910688
3,Apple,1.0,0.018343,1.0,item,0.000000,0.000000,1.834291,0.000000
4,Apple,1.0,0.016087,1.0,work,0.000000,0.000000,1.608671,0.000000
5,Apple,1.0,0.015329,1.0,days,0.000000,0.000000,1.532908,0.000000
6,Apple,1.0,0.015156,1.0,return,0.000000,0.000000,1.515596,0.000000
7,Apple,1.0,0.014359,1.0,didn,0.000000,0.000000,1.435890,0.000000
8,Apple,1.0,0.013649,1.0,seller,0.000000,0.000000,1.364880,0.000000
9,Apple,1.0,0.011793,1.0,lost,0.000000,1.179276,0.000000,-0.375246


In [229]:
topics = pd.unique(df['topic_id'])
brands = pd.unique(df['brand'])
overall = pd.unique(df['overall'])

In [230]:
after_filtering = pd.DataFrame(data=df)

# SKIP ALL TOPICS THAT NOT HAVE FEATURE WORD
for brand in brands:
    for topic in topics:
        for rate in overall:
            temp_df = pd.DataFrame(df.query('brand == @brand & topic_id == @topic & overall == @rate')).reset_index(drop=True)

            in_feature_word = False
            for row_num in range(0, len(temp_df), 1):

                word = temp_df['word'][row_num]

                if word in feature_words:
                    in_feature_word = True
                    break
                else:
                    in_feature_word = False

            if not in_feature_word:
                useless_topics = df[
                    (df['topic_id'] == topic) & (df['brand'] == brand) & (df['overall'] == rate)].index.tolist()
                after_filtering = after_filtering.drop(useless_topics, axis=0)


In [231]:
after_filtering

,brand,overall,score,topic_id,word,pos_score,neg_score,neu_score,comp_score
10,Apple,1.0,0.043710,2.0,use,0.000000,0.000000,4.370994,0.000000
11,Apple,1.0,0.026066,2.0,got,0.000000,0.000000,2.606556,0.000000
12,Apple,1.0,0.023646,2.0,sim,0.000000,0.000000,2.364608,0.000000
13,Apple,1.0,0.021570,2.0,work,0.000000,0.000000,2.157008,0.000000
14,Apple,1.0,0.020737,2.0,screen,0.000000,0.000000,0.000000,0.000000
15,Apple,1.0,0.020354,2.0,unlocked,0.000000,0.000000,2.035400,0.000000
16,Apple,1.0,0.017344,2.0,like,1.734410,0.000000,0.000000,0.626469
17,Apple,1.0,0.017343,2.0,broken,0.000000,1.734333,0.000000,-0.826756
18,Apple,1.0,0.017233,2.0,just,0.000000,0.000000,1.723255,0.000000
19,Apple,1.0,0.016126,2.0,company,0.000000,0.000000,1.612579,0.000000


In [232]:

result = []

# FOR EACH FEATURE WORD WE SUM THE SCORE OF ALL THE WORDS IN THE TOPIC
for brand in brands:
    for topic in topics:
        for rate in overall:
            temp_df = pd.DataFrame(after_filtering.query('brand == @brand & topic_id == @topic & overall == @rate')).reset_index(drop=True)

            temp_df_wo_ftr = pd.DataFrame(temp_df[~temp_df['word'].isin(feature_words)]).reset_index(drop=True)
            temp_df_w_ftr = pd.DataFrame(temp_df[temp_df['word'].isin(feature_words)]).reset_index(drop=True)

            pos = temp_df_wo_ftr['pos_score'].sum()
            neg = temp_df_wo_ftr['neg_score'].sum()
            neu = temp_df_wo_ftr['neu_score'].sum()
            comp = temp_df_wo_ftr['comp_score'].sum()

            for row_num in range(0, len(temp_df_w_ftr), 1):

                word = temp_df_w_ftr['word'][row_num]
                append_list = [brand, word, 1, pos, neg, neu, comp]
                result.append(append_list)
            
           

In [236]:
result = pd.DataFrame(data=result,
                      columns=["Brand", "Feature", "Density", "Positive", "Negative", "Neutral", "Compound"])

In [237]:
results_grouped = pd.DataFrame(data=result.groupby(["Brand", "Feature"]).sum())

In [238]:
results_grouped

Density    Positive  Negative     Neutral   Compound
Brand   Feature                                                      
Apple   apps           5   23.455182  1.637371   58.677850   9.169343
        battery       15   52.932238  3.211864  175.778655  20.979318
        camera         2    3.800905  1.849182   25.592165   0.930732
        price          3   25.375164  0.000000   36.833334  12.235748
        screen        18   76.536679  5.235690  211.742792  31.969068
LG      apps          10   42.370038  1.381107   93.324631  17.907126
        battery       28  119.338823  2.873023  270.039502  54.088027
        camera         9   43.266020  0.000000   76.248635  18.617050
        charge         1    5.670794  0.000000    7.211514   2.288760
        memory         1    3.324137  0.000000    7.621868   1.724090
        price          7   54.962405  0.000000   48.298768  25.757424
        screen        39  182.084332  2.873023  396.696608  83.144734
Samsung apps           6   32.289481  0.000000   59.003080  14.813944
        battery       29  124.785283  5.802105  301.609661  54.673559
        camera         4   24.914050  2.380207   36.832340  11.278040
        charge         1    3.235042  1.525077    9.740062   0.687383
        price          2   15.242763  0.000000   15.699218   6.948476
        screen        28  126.718144  4.431607  291.634219  58.180513